Notebook de gestion de la table de paramétrage des chargements. La table est dans le storage account et nécessite un sas token. 

La librairie "azure-cosmosdb-table" doit être installée dans le cluster.

In [0]:
from azure.cosmosdb.table.tableservice import TableService
from azure.cosmosdb.table.models import Entity
from datetime import datetime
from typing import Optional

TABLE_SAS_TOKEN = dbutils.secrets.get(scope="scope-datalake", key="storage-tables-sas-secret")
PARAMETERS_TABLE = 'LoadingParameters'

def LIN_create_table_if_not_exists(storage_account: str, table_name: str):
  # Créer la table dans le storage account si elle n'existe pas.
  # storage_account (str), storage account contenant la table.
  # table_name (str), nom de la table.
  table_service = TableService(account_name=storage_account, sas_token=TABLE_SAS_TOKEN)
  if not table_service.exists(table_name):
    table_service.create_table(table_name)
  
def LIN_set_loading_parameter(storage_account: str, parameter_name: str, value: datetime):
  # Définir un paramètre dans la table.
  # storage_account (str), storage account contenant la table.
  # parameter_name (str), nom du paramètre.
  # value (datetime), valeur du paramètre.
  table_service = TableService(account_name=storage_account, sas_token=TABLE_SAS_TOKEN)
  entity = {'PartitionKey': parameter_name.replace('/', '_'), 'RowKey': parameter_name.replace('/', '_'), 'value': value}
  table_service.insert_or_replace_entity(PARAMETERS_TABLE, entity)
  
def LIN_get_loading_parameter(storage_account: str, parameter_name: str) -> Optional[datetime]: 
  # Retourner la valeur d'un paramètre de la table.
  # storage_account (str), storage account contenant la table.
  # parameter_name (str), nom du paramètre.
  # retourne la valeur ou None.
  try:
    table_service = TableService(account_name=storage_account, sas_token=TABLE_SAS_TOKEN)
    entity = table_service.get_entity(PARAMETERS_TABLE, parameter_name.replace('/', '_'), parameter_name.replace('/', '_'))
    return entity.value
  except:
    return None
  
def LIN_delete_loading_parameter(storage_account: str, parameter_name: str):
  # Supprimer un paramètre de la table.
  # storage_account (str), storage account contenant la table.
  # parameter_name (str), nom du paramètre.
  try:
    table_service = TableService(account_name=storage_account, sas_token=TABLE_SAS_TOKEN)
    table_service.delete_entity(PARAMETERS_TABLE, parameter_name.replace('/', '_'), parameter_name.replace('/', '_'))
  except:
    pass